# Tableaux de bord

Ce notebook propose des visualisations avancées et des indicateurs de performance pour le projet BI.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Créer le dossier d'export si nécessaire
os.makedirs('../generated/graphs', exist_ok=True)
#Volume par type de matière genéré
faits_matieres = pd.read_csv('../generated/faits_matieres.csv')

# Conversion de la colonne générée en numérique (si ce n'est pas déjà fait)
if 'quantite_generee_donnees_agglo_num' not in faits_matieres.columns:
    faits_matieres['quantite_generee_donnees_agglo_num'] = (
        faits_matieres['quantite_generee_donnees_agglo']
        .astype(str)
        .str.replace(r'[^\d.,]', '', regex=True)
        .str.replace('\xa0', '', regex=False)
        .str.replace(',', '', regex=False)
    )
    faits_matieres['quantite_generee_donnees_agglo_num'] = pd.to_numeric(
        faits_matieres['quantite_generee_donnees_agglo_num'], errors='coerce'
    )

# Conversion de la colonne collectée en numérique (comme pour la colonne générée)
faits_matieres['quantite_collectee_donnees_agglo_num'] = (
    faits_matieres['quantite_collectee_donnees_agglo']
    .astype(str)
    .str.replace(r'[^\d.,]', '', regex=True)
    .str.replace('\xa0', '', regex=False)
    .str.replace(',', '', regex=False)
)
faits_matieres['quantite_collectee_donnees_agglo_num'] = pd.to_numeric(
    faits_matieres['quantite_collectee_donnees_agglo_num'], errors='coerce'
)

In [ ]:
# Croisement type de matière x territoire (producteur) x volume
import numpy as np
pivot = faits_matieres.pivot_table(
    index='matiere',
    columns='territoire',
    values='quantite_generee_donnees_agglo',
    aggfunc='sum',
    fill_value=0
)

# Croisement type de matière x territoire (producteur) x volume collecté
pivot = faits_matieres.pivot_table(
    index='matiere',
    columns='territoire',
    values='quantite_generee_donnees_agglo_num',
    aggfunc='sum',
    fill_value=0
)

plt.figure(figsize=(18,8))
sns.heatmap(pivot, annot=False, cmap='YlGnBu', linewidths=0.5)
plt.title('Volume collecté par type de matière et territoire (producteur)')
plt.xlabel('Territoire (producteur)')
plt.ylabel('Type de matière')
plt.tight_layout()
plt.savefig('../generated/graphs/heatmap_type_territoire_volume_collecte.png')
plt.show()

In [ ]:
# KPI : Volume total par producteur
# Utiliser la colonne numérique nettoyée pour l'agrégation
total_volume_producteur = faits_matieres.groupby('territoire')['quantite_collectee_donnees_agglo_num'].sum(min_count=1).sort_values(ascending=False)
print("Volume total par producteur :")
print(total_volume_producteur)

plt.figure(figsize=(10,5))
sns.barplot(x=[t[:20] + '…' if len(t) > 20 else t for t in total_volume_producteur.index], y=total_volume_producteur.values)
plt.xticks(rotation=45)
plt.title('Volume total par producteur')
plt.tight_layout()
plt.savefig('../generated/graphs/volume_par_producteur.png')
plt.show()






In [ ]:


# Agrégation par année en ignorant les valeurs nulles (inexistantes)
total_generee = faits_matieres.groupby('annee')['quantite_generee_donnees_agglo_num'].sum(min_count=1)
total_collectee = faits_matieres.groupby('annee')['quantite_collectee_donnees_agglo_num'].sum(min_count=1)

# Fusion dans un DataFrame
df_periode = pd.DataFrame({
    'Quantité générée': total_generee,
    'Quantité collectée': total_collectee
}).sort_index(ascending=True)

print(df_periode)

# Visualisation
plt.figure(figsize=(10,5))
df_periode.plot(kind='bar')
plt.title('Volume total généré vs collecté par période')
plt.ylabel('Tonnes')
plt.tight_layout()
plt.savefig('../generated/graphs/volume_genere_vs_collecte_par_periode.png')
plt.show()

In [ ]:
# KPI : Top 5 types de matières par volume
top5_matieres = faits_matieres.groupby('matiere')['quantite_generee_donnees_agglo_num'].sum().sort_values(ascending=False).head(5)
print("Top 5 types de matières par volume :")
print(top5_matieres)

plt.figure(figsize=(8,4))
sns.barplot(x=top5_matieres.index, y=top5_matieres.values)
plt.title('Top 5 types de matières par volume')
plt.tight_layout()
plt.savefig('../generated/graphs/top5_types_matiere.png')
plt.show()


In [ ]:
# Évolution du volume par type de matière au fil des années
plt.figure(figsize=(12,6))
sns.lineplot(data=faits_matieres, x='annee', y='quantite_generee_donnees_agglo', hue='matiere', marker='o')
plt.title('Évolution du volume par type de matière')
plt.tight_layout()
plt.savefig('../generated/graphs/evolution_volume_type_matiere.png')
plt.show()






In [ ]:
# Répartition (%) par type de matière (pie chart)
# Nettoyage et conversion en numérique
faits_matieres['quantite_generee_donnees_agglo_num'] = (
	faits_matieres['quantite_generee_donnees_agglo']
	.astype(str)
	.str.replace(r'[^\d.,]', '', regex=True)
	.str.replace('\xa0', '', regex=False)  # remove non-breaking spaces
	.str.replace(',', '', regex=False)     # remove commas if used as thousands separator
)
faits_matieres['quantite_generee_donnees_agglo_num'] = pd.to_numeric(
	faits_matieres['quantite_generee_donnees_agglo_num'], errors='coerce'
)

volumes = faits_matieres.groupby('matiere')['quantite_generee_donnees_agglo_num'].sum()
volumes = volumes[volumes > 0]  # Optionally, remove zero or NaN values

plt.figure(figsize=(8,8))
plt.pie(volumes, labels=volumes.index, autopct='%1.1f%%', startangle=140)
plt.title('Répartition du volume par type de matière')
plt.savefig('../generated/graphs/repartition_volume_type_matiere.png')
plt.show()


In [ ]:
# Analyse des valeurs manquantes et outliers
print('Valeurs manquantes par colonne:')
print(faits_matieres.isnull().sum())
plt.figure(figsize=(10,4))
sns.boxplot(data=faits_matieres, x='matiere', y='quantite_generee_donnees_agglo')
plt.xticks(rotation=45)
plt.title('Détection des outliers par type de matière')
plt.tight_layout()
plt.savefig('../generated/graphs/outliers_type_matiere.png')
plt.show()

# Tableau interactif (si Jupyter/IPython)
from IPython.display import display
print('Aperçu interactif des faits:')
display(faits_matieres.head(20))